<a href="https://colab.research.google.com/github/BluBloos/QMIND2021-2022/blob/main/src/HandTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

In [ ]:
# RUN THIS BLOCK ONLY WHEN IN COLAB

!echo "Initializing github repository"
!ls -la
!rm -r .config/
!rm -r sample_data/
!git clone https://github.com/BluBloos/QMIND2021-2022/ .

In [ ]:
# ALWAYS RUN THIS BLOCK, COLAB OR NOT

# Download updated project from Github.
!git pull

##### HANDLE DIFFS WHEN RUNNING IN COLAB #####
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
print("In Colab:", IN_COLAB)
import sys
if (IN_COLAB):
  sys.path.insert(1, '/content/src/')
##### HANDLE DIFFS WHEN RUNNING IN COLAB #####

########### TEST GPU AND RAM OF COLLAB INSTANCE ###########
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')
########### TEST GPU AND RAM OF COLLAB INSTANCE ###########

######### EXTERNAL LIBRARIES #########
import os
import pickle
import matplotlib.pyplot as plt
import imageio
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, UpSampling2D, MaxPool2D
from tensorflow.keras import Model
print("TensorFlow version:", tf.__version__)
#NOTE: Good resource. -> https://www.tensorflow.org/tutorials/quickstart/advanced
import cv2 # opencv, for image resizing.
######### EXTERNAL LIBRARIES #########

############## HELPER FUNCTIONS ############## 
# NOTE(Noah): Stole this function from Stackoverflow :)
def rgb2gray(rgb):
    return np.expand_dims(np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]), axis=2)
def resize(img, size):
    return cv2.resize(img, dsize=(size, size), interpolation=cv2.INTER_CUBIC)
############## HELPER FUNCTIONS ############## 

# MODEL LOADING

In [ ]:
# TODO(Noah): Get the MANO folders hosted in GCS so that this works again.
#   we note that this cost was tested and is in full working order, so 
#   the only thing not working is the lack of existence of MANO_DIR. 

# Setup some params.
IMAGE_SIZE = 224
GRAYSCALE = False
IMAGE_CHANNELS = 1 if GRAYSCALE else 3
BATCH_SIZE = 32
MANO_DIR = "mano_v1_2" if IN_COLAB else "../mano_v1_2"

from mobilehand import MAKE_MOBILE_HAND
from mobilehand_lfuncs import LOSS_3D

MOBILE_HAND = MAKE_MOBILE_HAND(IMAGE_SIZE, IMAGE_CHANNELS, BATCH_SIZE, MANO_DIR)

# INTEGRATION TEST
input_test = tf.random.uniform(shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
input_test = tf.cast(input_test, tf.float32)
output_test = MOBILE_HAND(input_test)
print(output_test)

# The lower training loop assumes that the model is set as such.
model = MOBILE_HAND

# The lower training loop also assumes that we have the loss function set like so.
loss_fn = lambda pred, gt : LOSS_3D(pred,gt) 

# DATA LOADING

In [ ]:
# more params to setup.
TRAIN_AMOUNT = 192 # must be multiple of 32
TEST_AMOUNT = 96 # must be multiple of 32

# Check if the dataset has been parsed yet. If not, parse.
data_dir = 'RHD_small' if IN_COLAB else '../RHD_small'
parsed_data_dir = 'SH_RHD' if IN_COLAB else '../SH_RHD'
import parsing_data
if not os.path.isdir(parsed_data_dir):
  os.mkdir(parsed_data_dir)
  os.mkdir(os.path.join(parsed_data_dir, 'evaluation'))
  os.mkdir(os.path.join(parsed_data_dir, 'evaluation', 'color'))
  os.mkdir(os.path.join(parsed_data_dir, 'training'))
  os.mkdir(os.path.join(parsed_data_dir, 'training', 'color'))
  parsing_data.parse_dataset(203, data_dir, parsed_data_dir)

data_dir = parsed_data_dir

# Load in the testing and training images.
x_train = np.zeros( (TRAIN_AMOUNT, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS) )
y_train = np.zeros( (TRAIN_AMOUNT, 21, 3) )
x_test = np.zeros( (TEST_AMOUNT, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS) ) 
y_test = np.zeros( (TEST_AMOUNT, 21, 3) )

def LoadData(dataAmount, dataType, np1):
  path = os.path.join(data_dir, dataType, 'color')
  
  count = 0
  for filename in os.listdir(path):
    filePath = os.path.join(path, filename)
    image = imageio.imread(filePath)
    _image = image.astype('float32')
    if GRAYSCALE:
      _image = rgb2gray(_image / 255)
    else:
      _image = _image / 255
    _image = resize(_image, IMAGE_SIZE)
    np1[count, :, :, :] = _image
    count += 1
    if (count >= dataAmount):
      break

print("Loading in the training data samples...")
start_time = time.time()
LoadData(TRAIN_AMOUNT, 'training', x_train)
x_train = x_train.astype('float32')
#y_train = y_train.astype('float32')
end_time = time.time()
print('Elapsed for LoadData training', end_time - start_time, 's')

print("Loading in the evaluation data samples...")
start_time = time.time()
LoadData(TEST_AMOUNT, 'evaluation', x_test)
x_test = x_test.astype('float32')
#y_test = y_test.astype('float32')
end_time = time.time()
print('Elapsed for LoadData evaluation', end_time - start_time, 's')

# Test print one of the images from the dataset.
_test = x_train[0] 
plt.imshow(_test)
plt.show()
#_test = y_train[0]
#plt.imshow(np.squeeze(_test))
#plt.show()

# Batch the data for tensorflow.
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# TODO(Noah): Is it feasible to now delete all the intermediate numpy arrays?
# TODO(Noah): Integrate code that max wrote to check if the dataset has been parsed.
# if the dataset has not been parsed yet, parse it, then load.

# TRAINING LOOP

In [ ]:
class StupidSimpleLossMetric():
    def __init__(self):
        self.losses = [] # empty python array 
    def __call__(self, loss):
        self.losses.append(loss)
    def result(self):
        return sum(self.losses) / len(self.losses)
    def reset_states(self):
        self.losses = []

optimizer = tf.keras.optimizers.Adam() # defaults should work just fine
train_loss = StupidSimpleLossMetric()
test_loss = StupidSimpleLossMetric()

# loss_fn = keras.losses.BinaryCrossentropy(from_logits=False) # from_logits=False just means that the values are between zero and one (a probability).
# loss_fn = lambda x, y: tf.math.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(x, y))

# train_accuracy = tf.keras.metrics.MeanAbsolutePercentageError(name='train_accuracy')
# test_accuracy = tf.keras.metrics.MeanAbsolutePercentageError(name='test_accuracy')

# @tf.function Compiles a function into a callable TensorFlow graph.
# https://www.tensorflow.org/guide/intro_to_graphs
'''def custom_loss_func(pred, labels):
    #return tf.math.reduce_sum( tf.math.square(labels - tf.cast(pred,tf.float64)))
    # now trying pixel-wise cross-entropy loss
    return -tf.math.reduce_sum(tf.math.log(pred) * labels)
'''

# Loss function unit test
input = tf.zeros([1, 39])  # mock pred of all zeros
label = np.expand_dims(y_train[0], axis=0)
loss = loss_fn(input, label)
print('Loss for pred of all zeros', loss.numpy())
#loss2 = loss_fn(label, label)
#print('Loss for perfect prediction', loss2.numpy())
input2 = tf.ones([1, 39])
loss3 = loss_fn(input2, label)
print('Loss for pred of all ones', loss3.numpy())

@tf.function
def train_step(input, gt):
    with tf.GradientTape() as tape:
        predictions = model(input)
        #loss = loss_func(predictions, segmentation_masks)
        #loss = np.dot(tf.reshape(segmentation_masks, [102400], tf.reshape(predictions, [102400])
        loss = loss_fn(predictions, gt)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss
    #train_accuracy(labels, predictions)
  
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  return loss_fn(predictions, labels)
  #test_accuracy(labels, predictions)

In [ ]:
# TODO: Reimplement loading in the saved model weights
# model.load_weights(checkpoint_path)

EPOCHS = 10 # sure...

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  print("Epoch", epoch)
  start = time.time()
  train_loss.reset_states()
  #train_accuracy.reset_states()
  test_loss.reset_states()
  #test_accuracy.reset_states()

  for images, labels in train_ds:
    loss = train_step(images, labels)
    train_loss(loss.numpy())

  for test_images, test_labels in test_ds:
    loss = test_step(test_images, test_labels)
    test_loss(loss.numpy())

  end = time.time()

  print(
    f'Epoch {epoch + 1}, '
    f'Time {end-start} s'
    f'Loss: {train_loss.result()}, '
    f'Test Loss: {test_loss.result()}, '
  )

  # for each epoch, we want to show the 
  #pred = model( _image )
  #plt.imshow(np.squeeze(pred))
  #plt.show()

# Save the model parameters
# TODO: Make it such that model parameters are saved after x many epochs as opposed to however
#   many epochs the model will be trained in total
#save_dir = '/content/drive/My Drive'
#checkpoint_path = save_dir + "/cp-{epoch:04d}.ckpt"
#model.save('current_model.h5py',save_path)
#model.save_weights(checkpoint_path.format(epoch=40))